In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
import math
import timeit
import scipy

#import matplotlib.pyplot as plt
import CNet
import provider
import TR
#%matplotlib inline
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /device:GPU:0


In [2]:
BATCH_SIZE=8
MAX_EPOCH=1
learning_rate = 3e-5
channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4=64,128,256,512,2,64,128,256,512

def model_init_fn(inputs1,inputs2, is_training):
    model = None

    model = CNet.CentralNet(channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4)

    return model(inputs1,inputs2)

def optimizer_init_fn():
    optimizer = None

    optimizer = tf.train.GradientDescentOptimizer(learning_rate)

    return optimizer

TR.train_part34(model_init_fn, optimizer_init_fn,100,device,BATCH_SIZE)

W0930 10:37:15.801015 139676561807296 deprecation_wrapper.py:119] From /home/nj/文档/NJ/centralnet_code/TR.py:15: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0930 10:37:15.891801 139676561807296 deprecation_wrapper.py:119] From /home/nj/文档/NJ/centralnet_code/CNet.py:248: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 10:37:15.898178 139676561807296 deprecation_wrapper.py:119] From /home/nj/文档/NJ/centralnet_code/CNet.py:14: The name tf.layers.Conv2D is deprecated. Please use tf.compat.v1.layers.Conv2D instead.

W0930 10:37:15.900449 139676561807296 deprecation_wrapper.py:119] From /home/nj/文档/NJ/centralnet_code/CNet.py:24: The name tf.layers.MaxPooling2D is deprecated. Please use tf.compat.v1.layers.MaxPooling2D instead.

W0930 10:37:15.906658 139676561807296 deprecation_wrapper.py:119] From /home/nj/文档/NJ/centralnet_code/CNet.py:94: The name tf.layers.Dense is deprecated. Please use tf.c

Starting epoch 0


KeyboardInterrupt: 

In [3]:
from tensorflow.contrib.framework.python.framework import checkpoint_utils
var_list = checkpoint_utils.list_variables("log/model_best.ckpt")
for v in var_list:
    print(v)


('conv2d/bias', [64])
('conv2d/kernel', [3, 3, 3, 64])
('conv2d_1/bias', [64])
('conv2d_1/kernel', [3, 3, 64, 64])
('conv2d_10/bias', [512])
('conv2d_10/kernel', [3, 3, 512, 512])
('conv2d_11/bias', [512])
('conv2d_11/kernel', [3, 3, 512, 512])
('conv2d_12/bias', [512])
('conv2d_12/kernel', [3, 3, 512, 512])
('conv2d_2/bias', [128])
('conv2d_2/kernel', [3, 3, 64, 128])
('conv2d_3/bias', [128])
('conv2d_3/kernel', [3, 3, 128, 128])
('conv2d_4/bias', [256])
('conv2d_4/kernel', [3, 3, 128, 256])
('conv2d_5/bias', [256])
('conv2d_5/kernel', [3, 3, 256, 256])
('conv2d_6/bias', [256])
('conv2d_6/kernel', [3, 3, 256, 256])
('conv2d_7/bias', [512])
('conv2d_7/kernel', [3, 3, 256, 512])
('conv2d_8/bias', [512])
('conv2d_8/kernel', [3, 3, 512, 512])
('conv2d_9/bias', [512])
('conv2d_9/kernel', [3, 3, 512, 512])
('dense/kernel', [211200, 64])
('dense_1/kernel', [108800, 128])
('dense_10/bias', [1000])
('dense_10/kernel', [4096, 1000])
('dense_11/bias', [2])
('dense_11/kernel', [1000, 2])
('dense_